## Imports

In [1]:
import os
import re
import time
import itertools

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as datetime

from scipy import stats
from pyhive import presto
from datetime import datetime, timedelta

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

In [3]:
## Connection
connection = presto.connect(
        
        host='presto-gateway.serving.data.production.internal',
#     presto.processing.yoda.run
#     presto-gateway.processing.data.production.internal',
        port=80,
        protocol='http',
        catalog='hive',
        username='vishnu.suthar@rapido.bike'
)

In [130]:
cwd = os.getcwd()
print(cwd)
local_datasource = '/Users/Anand/Downloads/One_tap/'

print(local_datasource)

/Users/Anand/Downloads/One_tap
/Users/Anand/Downloads/One_tap/


## Datasets & Parameter

In [12]:
## Parameter 
current_date = datetime.now()
start_date = '20240819'
end_date = '20240821'

# Convert date strings to datetime objects
start_dt = datetime.strptime(start_date, '%Y%m%d')
end_dt = datetime.strptime(end_date, '%Y%m%d')
segment_date = end_dt.strftime('%Y-%m-%d')
print(start_date, ' to ' ,end_date)

20240819  to  20240821


### AMJ Gross Customers

In [15]:
data_query = f"""

WITH base as(
    select 
        customer_id
    from(
        select
            *,
            case when total_orders = 1 then 1 else 0 end as orders_1,
            case when total_orders > 1 and total_orders <= 4 then 1 else 0 end as "orders_2-4",
            case when total_orders > 4 then 1 else 0 end as "orders_4+"
        from(
            select 
                *,
                case when link_orders > 0 and auto_orders = 0 and cab_orders = 0 then 'Link_only'
                     when link_orders = 0 and auto_orders > 0 and cab_orders = 0 then 'Auto_only'
                     when link_orders = 0 and auto_orders = 0 and cab_orders > 0 then 'Cab_only'
                     else 'Others' end as customer_type,
                (link_orders+auto_orders+cab_orders) as total_orders
            from(
                select 
                    customer_id,    
                    count(case when service_obj_service_name = 'Link' then order_id end) as link_orders,
                    count(case when service_obj_service_name = 'Auto' then order_id end) as auto_orders,
                    count(case when service_obj_service_name = 'CabEconomy' then order_id end) as cab_orders
                from orders.order_logs_fact 
                where yyyymmdd between date_format(current_date - interval '91' day,'%Y%m%d') and date_format(current_date - interval '1' day,'%Y%m%d')
                    and modified_order_status = 'dropped'
                    and (spd_fraud_flag = false or spd_fraud_flag is null)
                    and service_obj_service_name in ('Link','Auto','CabEconomy')
                    and city_name = 'Bangalore'
                group by 1 
                )
            )
        where customer_type in ('Cab_only')
        )
    where "orders_4+" > 0 
)

,customer_base as (
    select 
        olf.customer_id,
        city_name,
        customer_obj_mobile customer_mobile,
        COUNT(DISTINCT order_id) gross_orders,
        count(DISTINCT case when order_status = 'dropped' then order_id end) as net_orders,
        approx_percentile(CASE WHEN modified_order_status IN ('OCARA', 'COBRA', 'COBRM') THEN (customer_cancelled_epoch / 1000 - order_requested_epoch / 1000) END, 0.5) AS TTC,
        count(DISTINCT case when order_status = 'dropped' then order_id end)/COUNT(DISTINCT order_id) as G2N,
        count(DISTINCT case when modified_order_status = 'OCARA' then order_id end) as  OCARA,
        count(DISTINCT case when modified_order_status = 'COBRA' then order_id end) as COBRA,
        count(DISTINCT case when modified_order_status = 'COBRM' then order_id end) as COBRM
    FROM orders.order_logs_fact as olf 
    join base 
        on olf.customer_id = base.customer_id
    where yyyymmdd between date_format(current_date - interval '91' day,'%Y%m%d') and date_format(current_date - interval '1' day,'%Y%m%d')
        and service_obj_service_name in ('Link','Auto','CabEconomy')
        and city_name = 'Bangalore'    
        
    GROUP BY 1,2,3
    
    )
    , segment AS (
    
    SELECT
        customer_id,
        taxi_ltr_segment ltr_segment,
        taxi_retention_segments retention_segment,
        taxi_lifetime_stage lifetime_stage,
        taxi_service_affinity service_affinity,
        gender_tag gender
    FROM datasets.iallocator_customer_segments
    WHERE run_date = date_format(current_date - interval '2' day,'%Y-%m-%d')
    )
    
    SELECT 
        customer_base.*,
        coalesce( ltr_segment, 'UNKNOWN')ltr_segment,
        coalesce( retention_segment, 'UNKNOWN') retention_segment,
        coalesce( lifetime_stage, 'UNKNOWN') lifetime_stage,
        coalesce( service_affinity, 'UNKNOWN') service_affinity,
        coalesce( gender, 'UNKNOWN') gender
    FROM customer_base
    LEFT JOIN segment ON customer_base.customer_id = segment.customer_id
"""

# Execute the query and get the result as a DataFrame
df_data = pd.read_sql(data_query, connection)

# df_data.to_csv( local_datasource + '/customer_data_{}_{}.csv'.format(start_date, end_date), index=False)
df_data.head()

,customer_id,city_name,customer_mobile,gross_orders,net_orders,TTC,G2N,OCARA,COBRA,COBRM,ltr_segment,retention_segment,lifetime_stage,service_affinity,gender
0,663ad0454720ce3c573c5aad,Bangalore,9952118454,9,8,121.0,0,0,1,0,PHH,GOLD,SOFT_CHURN,UNKNOWN,MALE
1,63ddf7baf6ed681f53ab6cd1,Bangalore,9986699070,9,7,628.0,0,1,1,0,PHH,GOLD,HOOK,UNKNOWN,MALE
2,6346d7008217e7e292f02986,Bangalore,8073079727,11,5,277.0,0,3,1,0,PHH,DORMANT,DORMANT,UNKNOWN,MALE
3,66d00496d97e0e6e6023dbf9,Bangalore,6201114030,11,5,100.0,0,2,4,0,HH,HH,HANDHOLDING,UNKNOWN,MALE
4,62258b7df0b26b025a6352b2,Bangalore,9148757911,6,5,24.0,0,0,1,0,PHH,GOLD,HOOK,UNKNOWN,FEMALE


In [16]:
df_data['gender'].value_counts()

gender
MALE       4660
FEMALE     2754
UNKNOWN     125
OTHERS       21
Name: count, dtype: int64

In [17]:
# df_data = pd.read_csv( local_datasource + '/analysis_data_{}_{}.csv'.format(start_date, end_date))

In [18]:
df_data.head(5)

,customer_id,city_name,customer_mobile,gross_orders,net_orders,TTC,G2N,OCARA,COBRA,COBRM,ltr_segment,retention_segment,lifetime_stage,service_affinity,gender
0,663ad0454720ce3c573c5aad,Bangalore,9952118454,9,8,121.0,0,0,1,0,PHH,GOLD,SOFT_CHURN,UNKNOWN,MALE
1,63ddf7baf6ed681f53ab6cd1,Bangalore,9986699070,9,7,628.0,0,1,1,0,PHH,GOLD,HOOK,UNKNOWN,MALE
2,6346d7008217e7e292f02986,Bangalore,8073079727,11,5,277.0,0,3,1,0,PHH,DORMANT,DORMANT,UNKNOWN,MALE
3,66d00496d97e0e6e6023dbf9,Bangalore,6201114030,11,5,100.0,0,2,4,0,HH,HH,HANDHOLDING,UNKNOWN,MALE
4,62258b7df0b26b025a6352b2,Bangalore,9148757911,6,5,24.0,0,0,1,0,PHH,GOLD,HOOK,UNKNOWN,FEMALE


## User defined function

In [24]:
agg_dict = {
    'customers': ('customer_id', 'nunique'),
    'net_orders': ('net_orders', 'sum'),
    'gross_orders': ('gross_orders', 'sum'),
    'TTC': ('TTC', 'median'),
    'OCARA': ('OCARA', 'sum'),
    'COBRA': ('COBRA', 'sum'),
    'COBRM': ('COBRM', 'sum'),
}

def calculate_percentage(df, numerator, denominator):
    
    percentage = (df[numerator] * 100.00 / df[denominator]).round(2)
    
    return percentage

def calculate_percentage_columns(df):

    df['G2N'] = calculate_percentage(df, 'net_orders', 'gross_orders')
    df['OCARA'] = calculate_percentage(df, 'OCARA', 'net_orders')
    df['COBRA'] = calculate_percentage(df, 'COBRA', 'net_orders')
    df['COBRM'] = calculate_percentage(df, 'COBRM', 'net_orders')
    return df

## Sampling

In [27]:
df_data.head(2)

,customer_id,city_name,customer_mobile,gross_orders,net_orders,TTC,G2N,OCARA,COBRA,COBRM,ltr_segment,retention_segment,lifetime_stage,service_affinity,gender
0,663ad0454720ce3c573c5aad,Bangalore,9952118454,9,8,121.0,0,0,1,0,PHH,GOLD,SOFT_CHURN,UNKNOWN,MALE
1,63ddf7baf6ed681f53ab6cd1,Bangalore,9986699070,9,7,628.0,0,1,1,0,PHH,GOLD,HOOK,UNKNOWN,MALE


In [29]:
df_data.columns

Index(['customer_id', 'city_name', 'customer_mobile', 'gross_orders',
       'net_orders', 'TTC', 'G2N', 'OCARA', 'COBRA', 'COBRM', 'ltr_segment',
       'retention_segment', 'lifetime_stage', 'service_affinity', 'gender'],
      dtype='object')

In [31]:
df_analysis2 = df_data\
.groupby(['city_name'])\
.agg(**agg_dict)\
.reset_index()

df_analysis2 = calculate_percentage_columns(df_analysis2)
df_analysis2

,city_name,customers,net_orders,gross_orders,TTC,OCARA,COBRA,COBRM,G2N
0,Bangalore,7560,78968,136695,238.0,30.25,23.85,1.76,57.77


In [33]:
df_analysis_6 = df_data\
.groupby(['ltr_segment'])\
.agg(**agg_dict)\
.reset_index()

df_analysis_6 = calculate_percentage_columns(df_analysis_6)
df_analysis_6

,ltr_segment,customers,net_orders,gross_orders,TTC,OCARA,COBRA,COBRM,G2N
0,HH,48,254,406,212.0,23.23,26.77,3.15,62.56
1,PHH,7512,78714,136289,238.0,30.27,23.84,1.75,57.76


In [35]:
df_analysis_7 = df_data\
.groupby(['retention_segment'])\
.agg(**agg_dict)\
.reset_index()

df_analysis_7 = calculate_percentage_columns(df_analysis_7)
df_analysis_7

,retention_segment,customers,net_orders,gross_orders,TTC,OCARA,COBRA,COBRM,G2N
0,DORMANT,2254,16732,28385,242.5,28.75,24.01,2.16,58.95
1,ELITE,844,23887,40420,226.0,29.86,21.57,1.18,59.10
2,GOLD,2673,17394,30592,240.0,29.79,26.25,2.17,56.86
3,HH,47,248,400,212.0,23.79,27.42,3.23,62.00
4,PLATINUM,1704,20509,36523,242.0,32.41,24.17,1.75,56.15
5,PRIME,3,17,26,212.5,17.65,17.65,0.00,65.38
6,SILVER,35,181,349,216.0,30.39,38.67,0.00,51.86


In [37]:
df_analysis_8 = df_data\
.groupby(['lifetime_stage'])\
.agg(**agg_dict)\
.reset_index()

df_analysis_8 = calculate_percentage_columns(df_analysis_8)
df_analysis_8

,lifetime_stage,customers,net_orders,gross_orders,TTC,OCARA,COBRA,COBRM,G2N
0,CHURN_OTB,497,4728,7877,239.0,26.27,23.75,2.12,60.02
1,COMMITTED,1880,36029,62910,235.0,31.56,23.29,1.34,57.27
2,DETOX,1,6,7,2245.0,16.67,0.00,0.00,85.71
3,DORMANT,2254,16732,28385,242.5,28.75,24.01,2.16,58.95
4,HANDHOLDING,47,248,400,212.0,23.79,27.42,3.23,62.00
5,HOOK,1451,11184,17857,231.0,24.61,20.84,1.67,62.63
6,SOFT_CHURN,360,2659,3976,252.5,19.71,16.74,2.07,66.88
7,SUSTENANCE,1070,7382,15283,244.0,42.41,33.30,2.61,48.30


In [39]:
# df_analysis_9 = df_data\
# .groupby(['service_affinity'])\
# .agg(**agg_dict)\
# .reset_index()

# df_analysis_9 = calculate_percentage_columns(df_analysis_9)
# df_analysis_9

In [41]:
df_analysis_10 = df_data\
.groupby(['gender'])\
.agg(**agg_dict)\
.reset_index()

df_analysis_10 = calculate_percentage_columns(df_analysis_10)
df_analysis_10

,gender,customers,net_orders,gross_orders,TTC,OCARA,COBRA,COBRM,G2N
0,FEMALE,2754,33212,61317,245.0,34.18,25.94,1.34,54.16
1,MALE,4660,44445,73210,232.0,27.48,22.34,2.09,60.71
2,OTHERS,21,211,344,148.0,25.59,25.12,0.95,61.34
3,UNKNOWN,125,1100,1824,305.5,24.64,21.36,1.00,60.31


In [43]:
df_analysis_11 = df_data\
.groupby(['city_name','ltr_segment'])\
.agg(**agg_dict)\
.reset_index()

df_analysis_11 = calculate_percentage_columns(df_analysis_11)
df_analysis_11

,city_name,ltr_segment,customers,net_orders,gross_orders,TTC,OCARA,COBRA,COBRM,G2N
0,Bangalore,HH,48,254,406,212.0,23.23,26.77,3.15,62.56
1,Bangalore,PHH,7512,78714,136289,238.0,30.27,23.84,1.75,57.76


In [45]:
df_analysis_15 = df_data\
.groupby(['city_name', 'retention_segment'])\
.agg(**agg_dict)\
.reset_index()

df_analysis_15 = calculate_percentage_columns(df_analysis_15)
df_analysis_15

,city_name,retention_segment,customers,net_orders,gross_orders,TTC,OCARA,COBRA,COBRM,G2N
0,Bangalore,DORMANT,2254,16732,28385,242.5,28.75,24.01,2.16,58.95
1,Bangalore,ELITE,844,23887,40420,226.0,29.86,21.57,1.18,59.10
2,Bangalore,GOLD,2673,17394,30592,240.0,29.79,26.25,2.17,56.86
3,Bangalore,HH,47,248,400,212.0,23.79,27.42,3.23,62.00
4,Bangalore,PLATINUM,1704,20509,36523,242.0,32.41,24.17,1.75,56.15
5,Bangalore,PRIME,3,17,26,212.5,17.65,17.65,0.00,65.38
6,Bangalore,SILVER,35,181,349,216.0,30.39,38.67,0.00,51.86


In [47]:
df_data['city_name'].value_counts(True)

city_name
Bangalore    1.0
Name: proportion, dtype: float64

In [50]:
df_data.shape

(7560, 15)

In [52]:
df_data.customer_id.nunique()

7560

In [54]:
# duplicated_values = df_data['customer_id'].duplicated(keep=False) 
# df_cleaned = df_data[~duplicated_values]
df_cleaned = df_data
df_cleaned.shape

(7560, 15)

In [56]:
df_cleaned.customer_id.nunique()

7560

In [58]:
random_seed = 42

df_train_control, df_test = train_test_split(df_cleaned, 
                                             test_size=0.5, 
                                             random_state=random_seed
                                            )

df_train_control = df_train_control.sample(len(df_train_control), random_state=random_seed)
df_test = df_test.sample(len(df_test), random_state=random_seed)

# Assign group labels
df_train_control['group_tc'] = 'Control'
df_test['group_tc'] = 'Test'

df_final_sample = pd.concat([df_train_control, df_test])

In [60]:
df_final_sample['group_tc'].value_counts(True)

group_tc
Control    0.5
Test       0.5
Name: proportion, dtype: float64

In [62]:
df_final_sample.groupby('city_name')['group_tc'].value_counts(True).unstack(fill_value=0)

group_tc,Control,Test
city_name,,
Bangalore,0.5,0.5


In [64]:
df_final_sample.columns

Index(['customer_id', 'city_name', 'customer_mobile', 'gross_orders',
       'net_orders', 'TTC', 'G2N', 'OCARA', 'COBRA', 'COBRM', 'ltr_segment',
       'retention_segment', 'lifetime_stage', 'service_affinity', 'gender',
       'group_tc'],
      dtype='object')

In [66]:
df1 = df_final_sample\
.groupby(['city_name','group_tc'])\
.agg(**agg_dict)\
.reset_index()

df1 = calculate_percentage_columns(df1)
df1

,city_name,group_tc,customers,net_orders,gross_orders,TTC,OCARA,COBRA,COBRM,G2N
0,Bangalore,Control,3780,40558,71398,240.0,30.70,24.69,1.90,56.81
1,Bangalore,Test,3780,38410,65297,236.0,29.78,22.95,1.61,58.82


In [68]:
df2 = df_final_sample\
.groupby(['city_name','ltr_segment', 'group_tc'])\
.agg(**agg_dict)\
.reset_index()

df2 = calculate_percentage_columns(df2)
df2

,city_name,ltr_segment,group_tc,customers,net_orders,gross_orders,TTC,OCARA,COBRA,COBRM,G2N
0,Bangalore,HH,Control,28,154,244,186.0,22.08,30.52,1.95,63.11
1,Bangalore,HH,Test,20,100,162,242.5,25.00,21.00,5.00,61.73
2,Bangalore,PHH,Control,3752,40404,71154,240.0,30.73,24.67,1.90,56.78
3,Bangalore,PHH,Test,3760,38310,65135,236.0,29.79,22.96,1.60,58.82


In [70]:
df3 = df_final_sample\
.groupby(['city_name','retention_segment', 'group_tc'])\
.agg(**agg_dict)\
.reset_index()

df3 = calculate_percentage_columns(df3)
df3

,city_name,retention_segment,group_tc,customers,net_orders,gross_orders,TTC,OCARA,COBRA,COBRM,G2N
0,Bangalore,DORMANT,Control,1114,8096,13966,245.0,28.66,25.49,1.98,57.97
1,Bangalore,DORMANT,Test,1140,8636,14419,241.0,28.83,22.61,2.33,59.89
2,Bangalore,ELITE,Control,437,13373,23343,230.0,30.45,23.54,1.57,57.29
3,Bangalore,ELITE,Test,407,10514,17077,222.0,29.11,19.06,0.69,61.57
4,Bangalore,GOLD,Control,1356,8836,15766,237.0,30.67,27.22,1.99,56.04
5,Bangalore,GOLD,Test,1317,8558,14826,244.0,28.89,25.25,2.36,57.72
6,Bangalore,HH,Control,27,148,238,186.0,22.97,31.76,2.03,62.18
7,Bangalore,HH,Test,20,100,162,242.5,25.00,21.00,5.00,61.73
8,Bangalore,PLATINUM,Control,827,10007,17868,249.0,32.73,23.06,2.20,56.01
9,Bangalore,PLATINUM,Test,877,10502,18655,235.0,32.11,25.21,1.31,56.30


In [72]:
df4 = df_final_sample\
.groupby(['city_name','lifetime_stage', 'group_tc'])\
.agg(**agg_dict)\
.reset_index()

df4 = calculate_percentage_columns(df4)
df4

,city_name,lifetime_stage,group_tc,customers,net_orders,gross_orders,TTC,OCARA,COBRA,COBRM,G2N
0,Bangalore,CHURN_OTB,Control,241,2336,3869,269.0,26.84,22.73,1.76,60.38
1,Bangalore,CHURN_OTB,Test,256,2392,4008,227.0,25.71,24.75,2.47,59.68
2,Bangalore,COMMITTED,Control,935,19306,34154,237.5,31.45,24.29,1.70,56.53
3,Bangalore,COMMITTED,Test,945,16723,28756,232.0,31.69,22.13,0.93,58.15
4,Bangalore,DETOX,Control,1,6,7,2245.0,16.67,0.00,0.00,85.71
5,Bangalore,DORMANT,Control,1114,8096,13966,245.0,28.66,25.49,1.98,57.97
6,Bangalore,DORMANT,Test,1140,8636,14419,241.0,28.83,22.61,2.33,59.89
7,Bangalore,HANDHOLDING,Control,27,148,238,186.0,22.97,31.76,2.03,62.18
8,Bangalore,HANDHOLDING,Test,20,100,162,242.5,25.00,21.00,5.00,61.73
9,Bangalore,HOOK,Control,738,5607,8898,230.0,23.54,19.71,1.91,63.01


In [74]:
df_final_sample.groupby('city_name')['group_tc'].value_counts(True).unstack(fill_value=0)

group_tc,Control,Test
city_name,,
Bangalore,0.5,0.5


In [76]:
df_final_sample.groupby(['city_name', 'ltr_segment'])['group_tc'].value_counts(True).unstack(fill_value=0)

group_tc                Control      Test
city_name ltr_segment                    
Bangalore HH           0.583333  0.416667
          PHH          0.499468  0.500532

In [78]:
df_final_sample.groupby(['city_name', 'retention_segment'])['group_tc'].value_counts(True).unstack(fill_value=0)

group_tc                      Control      Test
city_name retention_segment                    
Bangalore DORMANT            0.494232  0.505768
          ELITE              0.517773  0.482227
          GOLD               0.507295  0.492705
          HH                 0.574468  0.425532
          PLATINUM           0.485329  0.514671
          PRIME              0.333333  0.666667
          SILVER             0.514286  0.485714

### Check

In [81]:
df_final_sample.query("group_tc == 'Control'")["ltr_segment"].value_counts(True)

ltr_segment
PHH    0.992593
HH     0.007407
Name: proportion, dtype: float64

In [83]:
df_final_sample.query("group_tc == 'Test'")["ltr_segment"].value_counts(True)

ltr_segment
PHH    0.994709
HH     0.005291
Name: proportion, dtype: float64

In [85]:
df_final_sample.query("group_tc == 'Control'")["retention_segment"].value_counts(True)

retention_segment
GOLD        0.358730
DORMANT     0.294709
PLATINUM    0.218783
ELITE       0.115608
HH          0.007143
SILVER      0.004762
PRIME       0.000265
Name: proportion, dtype: float64

In [87]:
df_final_sample.query("group_tc == 'Test'")["retention_segment"].value_counts(True)

retention_segment
GOLD        0.348413
DORMANT     0.301587
PLATINUM    0.232011
ELITE       0.107672
HH          0.005291
SILVER      0.004497
PRIME       0.000529
Name: proportion, dtype: float64

### Export

In [90]:
df_final_sample.head(5)

,customer_id,city_name,customer_mobile,gross_orders,net_orders,TTC,G2N,OCARA,COBRA,COBRM,ltr_segment,retention_segment,lifetime_stage,service_affinity,gender,group_tc
631,667a830ea7577a17a568d523,Bangalore,8050287203,6,5,NaN,0,0,0,0,PHH,DORMANT,DORMANT,UNKNOWN,FEMALE,Control
1141,66b5f13406a89c1e331117fa,Bangalore,9663753523,33,28,354.0,0,4,0,0,PHH,ELITE,COMMITTED,ONLY_CAB,FEMALE,Control
7308,629b6a4692702e79004fbdcb,Bangalore,9967059144,12,6,152.0,0,1,3,0,PHH,DORMANT,DORMANT,UNKNOWN,MALE,Control
3177,66718b67212b3e3da0e36edb,Bangalore,9945979060,19,7,289.0,0,5,5,0,PHH,DORMANT,DORMANT,ONLY_CAB,MALE,Control
845,64e05e7d4c26f8239edfdcd8,Bangalore,7008902368,94,55,145.0,0,18,18,0,PHH,ELITE,COMMITTED,ONLY_CAB,FEMALE,Control


In [92]:
df_final_sample.group_tc.unique()

array(['Control', 'Test'], dtype=object)

In [94]:
# df_train_control['customer_mobile'] = df_train_control['customer_mobile'].astype(float).astype(int)

In [100]:
df_final_sample[df_final_sample['group_tc'] == 'Control'].to_csv(local_datasource +'Only_Cab' +'_Control_export.csv', index=False)

In [102]:
df_final_sample[df_final_sample['group_tc'] == 'Test'].to_csv(local_datasource + 'Only_Cab'+'_Test_export.csv', index=False)

In [104]:
df_export_test = df_final_sample[df_final_sample['group_tc'] == 'Test']

In [112]:
df_export_control = df_final_sample[df_final_sample['group_tc'] == 'Control']

In [106]:
df_export_test.shape

(3780, 16)

In [108]:
df_export_test.head(5)

,customer_id,city_name,customer_mobile,gross_orders,net_orders,TTC,G2N,OCARA,COBRA,COBRM,ltr_segment,retention_segment,lifetime_stage,service_affinity,gender,group_tc
2664,644234d9c79eb782c767165a,Bangalore,6363881461,8,5,243.0,0,3,0,0,PHH,GOLD,HOOK,UNKNOWN,UNKNOWN,Test
4094,62db8d3d7db0c2ca8afab728,Bangalore,9088889864,30,20,472.0,0,5,2,0,PHH,PLATINUM,COMMITTED,ONLY_CAB,MALE,Test
4323,65dc121dd1f0f1a33258cd00,Bangalore,8447781005,12,5,289.0,0,5,1,0,PHH,ELITE,COMMITTED,ONLY_CAB,FEMALE,Test
1255,621b5a02ac1f972693aeb6c2,Bangalore,6362840978,8,5,412.0,0,3,0,0,PHH,GOLD,HOOK,UNKNOWN,FEMALE,Test
2115,5b9b4d6d0dd8c472dd253ffc,Bangalore,9915259210,12,6,157.0,0,1,2,0,PHH,GOLD,COMMITTED,AUTO_CAB,MALE,Test


In [114]:
test_save_data = df_export_test[['customer_mobile']]
control_save_data = df_export_control[['customer_mobile']]

In [116]:
control_save_data

,customer_mobile
631,8050287203
1141,9663753523
7308,9967059144
3177,9945979060
845,7008902368
...,...
163,8147253053
7075,7411613832
5259,8884879666
6390,9899289529


In [132]:
control_save_data.to_csv(local_datasource + 'Only_Cab'+'_one_tap_control_customers.csv', index=False, header=False)

In [134]:
test_save_data.to_csv(local_datasource +'Only_Cab'+'_one_tap_test_customers.csv', index=False, header=False)

In [136]:
control_save_data

,customer_mobile
631,8050287203
1141,9663753523
7308,9967059144
3177,9945979060
845,7008902368
...,...
163,8147253053
7075,7411613832
5259,8884879666
6390,9899289529


In [138]:
test_save_data

,customer_mobile
2664,6363881461
4094,9088889864
4323,8447781005
1255,6362840978
2115,9915259210
...,...
5664,9741353874
2615,9150430710
2522,8015080650
4800,8847806520


In [82]:
len(test_save_data) // 3

42901

In [72]:
test_save_data = df_export_test[['customer_mobile']]

def df_split(test_save_data):
    # Determine chunk size
    chunk_size = len(test_save_data) // 3
    
    # Loop to split the DataFrame into 3 parts
    for i in range(3):
        start = i * chunk_size
        # Ensure the last chunk covers the remaining rows
        end = (i + 1) * chunk_size if i < 2 else len(test_save_data)
        
        # Extract the chunk
        chunk = test_save_data.iloc[start:end]
        
        # Save the chunk as a CSV file
        chunk.to_csv(f'test_data_{i+1}.csv', header=False, index=False)

        if saved_chunk.duplicated().any():
            print(f"Duplicates found in test_data_{i+1}.csv")
        else:
            print(f"No duplicates found in test_data_{i+1}.csv")


In [73]:
chunk.nunique()

NameError: name 'chunk' is not defined

In [74]:
df_split(test_save_data)
df_data.customer_id.nunique()

NameError: name 'saved_chunk' is not defined

In [ ]:
control_save_data = df_export_test[['customer_mobile']]

In [166]:
test_save_data.to_csv('/Users/rapido123/Downloads/Sampling Data/Local data/output.csv', index=False)

In [167]:
save_data.to_csv('/Users/rapido123/Downloads/Sampling Data/Local data/Test_output.csv', index=False)